In [ ]:
import cv2
import numpy as np
body_cascade = cv2.CascadeClassifier('/home/mini/Downloads/haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
data=[]
ix=0
while (1):
    ret,img = cap.read()
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = body_cascade.detectMultiScale(gray,1.3,5)
    for(x,y,w,h) in faces:
        face_component = img[y:y+h, x:x+w, :]
        fc = cv2.resize(face_component, (50, 50))
        if ix%10 == 0 and len(data) < 20:
            data.append(fc)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    ix+=1 
    cv2.imshow('img',img)
    if cv2.waitKey(1) == 27 or len(data) >= 20:
        break
cap.release()
cv2.destroyAllWindows()
data=np.asarray(data)
print(data.shape)
np.save('face_01',data)

In [ ]:
import cv2
import numpy as np
body_cascade = cv2.CascadeClassifier('/home/mini/Downloads/haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
data=[]
ix=0
while (1):
    ret,img = cap.read()
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = body_cascade.detectMultiScale(gray,1.3,5)
    for(x,y,w,h) in faces:
        face_component = img[y:y+h, x:x+w, :]
        fc = cv2.resize(face_component, (50, 50))
        if ix%10 == 0 and len(data) < 20:
            data.append(fc)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    ix+=1 
    cv2.imshow('img',img)
    if cv2.waitKey(1) == 27 or len(data) >= 20:
        break
cap.release()
cv2.destroyAllWindows()
data=np.asarray(data)
print(data.shape)
np.save('face_02',data)

In [12]:
import numpy as np
import cv2

# instantiate the camera object and haar cascade
cam = cv2.VideoCapture(0)
face_cas = cv2.CascadeClassifier('/home/mini/Downloads/haarcascade_frontalface_default.xml')

# declare the type of font to be used on output window
font = cv2.FONT_HERSHEY_SIMPLEX

# load the data from the numpy matrices and convert to linear vectors
f_01 = np.load('face_01.npy').reshape((20, 50*50*3))    # Prashant
f_02=  np.load('face_02.npy').reshape((20, 50*50*3))    # Nil

print(f_01.shape, f_02.shape)

# create a look-up dictionary
names = {
    0: 'Mini',
    1: 'Simran'

}

# create a matrix to store the labels
labels = np.zeros((40, 1))
labels[:20, :] = 0.0    
labels[20:, :] = 1.0 

# combine all info into one data array
data = np.concatenate([f_01, f_02]) # (40, 7500)
print(data.shape, labels.shape)  # (40, 1)

# the distance and knn functions we defined earlier
def distance(x1, x2):
    return np.sqrt(((x1-x2)**2).sum())

def knn(x, train, targets, k=5):
    m = train.shape[0]
    dist = []
    for ix in range(m):
        # compute distance from each point and store in dist
        dist.append(distance(x, train[ix]))
    dist = np.asarray(dist)
    indx = np.argsort(dist)
    sorted_labels = labels[indx][:k]
    counts = np.unique(sorted_labels, return_counts=True)
    return counts[0][np.argmax(counts[1])]

while True:
    # get each frame
    ret, frame = cam.read()

    if ret == True:
        # convert to grayscale and get faces
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cas.detectMultiScale(gray, 1.3, 5)

        # for each face
        for (x, y, w, h) in faces:
            face_component = frame[y:y+h, x:x+w, :]
            fc = cv2.resize(face_component, (50, 50))


            lab = knn(fc.flatten(), data, labels)
           
            text = names[int(lab)]

           
            cv2.putText(frame, text, (x, y), font, 1, (255, 255, 0), 2)

            
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.imshow('face recognition', frame)

        if cv2.waitKey(1) == ord('q'):
            break
    else:
        print('Error')

cv2.destroyAllWindows()

(20, 7500) (20, 7500)
(40, 7500) (40, 1)


In [13]:
cam.release()